In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
os.environ["KERAS_BACKEND"]="jax" 
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input,MobileNetV2
from sklearn.model_selection import train_test_split

In [3]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar100.load_data()

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [4]:
x_train=tf.image.resize(x_train,(128,128)).numpy()
x_test=tf.image.resize(x_test,(128,128)).numpy()

In [5]:
x_train=tf.keras.applications.mobilenet.preprocess_input(x_train)
x_test=tf.keras.applications.mobilenet.preprocess_input(x_test)

In [6]:
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,random_state=42,test_size=0.25,stratify=y_train)

In [7]:
datagen=ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

In [8]:
base_model=MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(128,128,3)
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
base_model.trainable=False

In [10]:
model=models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(512,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(256,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(100,activation='softmax') 
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_128 (Functional)    │ (None, 4, 4, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       1,311,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 100)                 │          25,700 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,258,724 (16.25 MB)

 Trainable params: 1,997,156 (7.62 MB)

 Non-trainable params: 2,261,568 (8.63 MB)

In [11]:
model.compile(optimizer=Adam(learning_rate=1e-4),loss='sparse_categorical_crossentropy',metrics=["accuracy"],jit_compile=True)

In [12]:
early_stopper=EarlyStopping(restore_best_weights=True,monitor="val_loss",patience=5)

In [13]:
history=model.fit(datagen.flow(x_train,y_train),epochs=30,batch_size=32,callbacks=[early_stopper],validation_data=[x_val,y_val],verbose=1)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 156s 129ms/step - accuracy: 0.0716 - loss: 4.6216 - val_accuracy: 0.4328 - val_loss: 2.2683
Epoch 2/30
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 138s 116ms/step - accuracy: 0.2698 - loss: 2.9800 - val_accuracy: 0.5128 - val_loss: 1.8302
Epoch 3/30
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 136s 116ms/step - accuracy: 0.3469 - loss: 2.5596 - val_accuracy: 0.5488 - val_loss: 1.6488
Epoch 4/30
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 136s 116ms/step - accuracy: 0.3935 - loss: 2.3298 - val_accuracy: 0.5652 - val_loss: 1.5584
Epoch 5/30
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 136s 116ms/step - accuracy: 0.4164 - loss: 2.2172 - val_accuracy: 0.5748 - val_loss: 1.5069
Epoch 6/30
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 135s 115ms/step - accuracy: 0.4393 - loss: 2.1234 - val_accuracy: 0.5908 - val_loss: 1.4535
Epoch 7/30
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 136s 116ms/step - accuracy: 0.4510 - loss: 2.0529 - val_accuracy: 0.6002 - val_loss: 1.4276
Epoch 8/30
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 135s 115ms/step - ac

In [14]:
import numpy as np
y_pred=model.predict(x_test)
y_pred_class=np.argmax(y_pred,axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step


In [15]:
test_loss,test_accuracy=model.evaluate(x_test,y_test)
print(f"Test accuracy: {test_accuracy * 100:.4f}%")

313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.6489 - loss: 1.2344
Test accuracy: 64.7400%


In [16]:
val_loss,val_accuracy=model.evaluate(x_val,y_val)
print(f"Validation accuracy: {val_accuracy * 100:.4f}%")

391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6437 - loss: 1.2521
Validation accuracy: 64.6640%


In [17]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score,classification_report


accuracy=accuracy_score(y_test,y_pred_class)
precision=precision_score(y_test,y_pred_class,average='weighted')
recall=recall_score(y_test,y_pred_class,average='weighted')
f1=f1_score(y_test,y_pred_class,average='weighted')


print("\nPerformance Metrics Breakdown for MobileNetV2:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


Performance Metrics Breakdown for MobileNetV2:
Accuracy: 0.6474
Precision: 0.6527
Recall: 0.6474
F1-Score: 0.6451


In [19]:
print("\nDetailed Classification Report for MobileNetBase:")
print(classification_report(y_test,y_pred_class))


Detailed Classification Report for MobileNetBase:
              precision    recall  f1-score   support

           0       0.87      0.90      0.89       100
           1       0.70      0.85      0.77       100
           2       0.67      0.49      0.57       100
           3       0.47      0.55      0.50       100
           4       0.33      0.50      0.40       100
           5       0.64      0.63      0.64       100
           6       0.75      0.58      0.66       100
           7       0.75      0.65      0.70       100
           8       0.79      0.75      0.77       100
           9       0.84      0.81      0.82       100
          10       0.53      0.51      0.52       100
          11       0.39      0.43      0.41       100
          12       0.75      0.71      0.73       100
          13       0.66      0.48      0.55       100
          14       0.58      0.69      0.63       100
          15       0.59      0.62      0.60       100
          16       0.77      0